<center><b>Deep Learning - Convolutional Network Ensemble (Boosting)</b></center><br>

The code below implements multiple convolutional networks and utilises them in an ensemble to improve prediction. The ensemble method used here is Boosting. Boosting affects how the data is selected for training. Each model only receives a subset of the training data. The first model to be trained gets a random selection with replacement of the data. The following models receive the training data that couldn't be correctly predicted by the previous model and some randomly selected data with replacement. The effectiveness of this method comes from the fact that each incorrectly predicted training data instance will be retrained on other models until a correct prediction is found. <br> <br>
The dataset is very small, it contains 17 categories but only 1020 images in total. To adjust for this, data augmentation is implemented also the data is only split into 'training' and 'validation' sets, 'test' sets are not being utilised.<br> <br>
Data checkpointing is also used that caches the best model with the smallest validation loss, this allows incremental training. Custom data checkpointing was used to compensate for a shortcoming of Keras' built-in data checkpointer as that system loses the information about the validation loss when loading the data back in. This custom checkpointer saves the validation loss into a separate file.<br>
<br>
As can be seen on the results, the ensemble method resulted in a better performance than any of the architectures did by themselves.

In [1]:
import os.path
import h5py
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
import matplotlib.pyplot as plt
import zipfile

import tensorflow as tf
print(tf.__version__)

Using TensorFlow backend.


2.1.0


In [2]:
# Handle dataset
FILE_PATH = os.getcwd()
CACHE_PATH = FILE_PATH+"/cached/deeplearning/"
data_file = FILE_PATH+"/data/DeepLearning/conv_net_data.zip"

data = zipfile.ZipFile(data_file)
data_file = data.open('data1.h5')

def loadDataH5():
    with h5py.File(data_file,'r') as hf:
      trainX = np.array(hf.get('trainX'))
      trainY = np.array(hf.get('trainY'))
      valX = np.array(hf.get('valX'))
      valY = np.array(hf.get('valY'))
      print (trainX.shape,trainY.shape)
      print (valX.shape,valY.shape)
      return trainX, trainY, valX, valY 
 
trainX, trainY, testX, testY = loadDataH5() 

(1020, 128, 128, 3) (1020,)
(340, 128, 128, 3) (340,)


In [3]:
# const variables about the dataset
IMG_DEPTH   = 3
IMG_WIDTH   = 128
IMG_HEIGHT  = 128
NUM_CLASSES = 17


NUM_EPOCHS  = 50

In [4]:
# All Architectures used

def create_case_0_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    # add conv layer
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

def create_case_1_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    
    # add conv layer 1
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    # add pooling layer 1
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 2
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 2
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

def create_case_2_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    
    # add conv layer 1
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    # add pooling layer 1
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 2
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 2
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 3
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 3
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

def create_case_3_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    
    # add conv layer 1
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    # add pooling layer 1
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 2
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 2
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 3
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 3
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 4
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 4
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(512, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

def create_alex_net_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    
    # add conv layer 1
    model.add(tf.keras.layers.Conv2D(96, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    # add pooling layer 1
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 2
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    # add pooling layer 2
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 3
    model.add(tf.keras.layers.Conv2D(384, (3, 3), padding="same", activation='relu'))
    # add conv layer 4
    model.add(tf.keras.layers.Conv2D(384, (3, 3), padding="same", activation='relu'))
    # add conv layer 5
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    # add pooling layer 3
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dense(1024, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

def create_vgg_16_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    
    # add 2 conv layers
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    
    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    
    # add 2 conv layers
    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation='relu'))
    
    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    
    # add 3 conv layers
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    
    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add 3 conv layers
    model.add(tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'))

    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dense(1024, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

In [5]:
# Base class for all architectures to derive from. Handles checkpointing and data augmentation
class CNN_base_augmented_checkpointed:
  def __init__(self, try_to_load_weights=True, ext_name=""):
    # Initialise Variables
    self.BATCH_SIZE = 16
  
    # Initialise model
    self.model = self.create_model(IMG_WIDTH, IMG_HEIGHT, IMG_DEPTH, NUM_CLASSES)
    self.model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), metrics=['accuracy'])

    # Saved data file names
    self.weights_file_name = "{}/{}{}.hdf5".format(CACHE_PATH, self.get_name(), ext_name)
    self.val_loss_file_name = "{}/{}{}_val_loss.txt".format(CACHE_PATH, self.get_name(), ext_name)

    # The current best val_loss
    self.best_val_loss = float("inf")
    if try_to_load_weights:
      self.load_checkpointed_weights()
    

  def load_best_val_loss_value(self):
    """
    This function is used to load the validation loss of the best model.
    It is stored in a txt file when the best model's weights were cached
    """
    if os.path.isfile(self.val_loss_file_name):
      loss_file = open(self.val_loss_file_name, "r")
      self.best_val_loss = float(loss_file.read())
      loss_file.close()
      print("{} - Loaded best val_loss value: {:.4f} from: {}".format(self.get_name(), self.best_val_loss, self.val_loss_file_name))


  def save_weights_callback(self, logs):
    """
    This is a custom callback function to be used to cache the weights of the network when val_loss decreases
    """
    val_loss = logs.get('val_loss')
    if val_loss < self.best_val_loss:
      print("\n{} - Caching Checkpoint - val_loss has improved from: {:.4f} to: {:.4f}".format(self.get_name(), self.best_val_loss, val_loss))
      self.best_val_loss = val_loss
      
      self.model.save_weights(self.weights_file_name)

      loss_file = open(self.val_loss_file_name,"w")
      loss_file.write(str(val_loss))
      loss_file.close()
    else:
      print("\n{} Val_loss did not improve. Best is: {:.4f}. Not Caching".format(self.get_name(), self.best_val_loss))

  def train(self, tr_x, tr_y, val_x, val_y, num_epochs=NUM_EPOCHS):
    print("{} - Training for {} epochs".format(self.get_name(), num_epochs))

    # Create the image generator for Data Augmentation
    datagen_batch = int(len(tr_x) / self.BATCH_SIZE)
    datagen = self.create_image_generator()

    # Create the model checkpointer
    custom_checkpointing_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs:self.save_weights_callback(logs))

    # Train
    self.model.fit(datagen.flow(tr_x, tr_y, batch_size=self.BATCH_SIZE), steps_per_epoch=datagen_batch, epochs=num_epochs, validation_data=(val_x, val_y), callbacks=[custom_checkpointing_callback])

  def load_checkpointed_weights(self):
    # If possible load the previous training results. Weights and assosiated val_loss
    if os.path.isfile(self.weights_file_name):
      print("{} - Loading previous training data".format(self.get_name()))
      self.model.load_weights(self.weights_file_name)
      print("{} - Loaded previous model weights from: {}".format(self.get_name(), self.weights_file_name))
      self.load_best_val_loss_value()

  def predict(self, val_x):
    return self.model.predict(val_x)

  def get_instances_incorrectly_predicted(self, tr_x, tr_y):
    # Get Prediction
    pred_y = self.predict(tr_x)
    # Calculate Accuracy
    pred_y = tf.math.argmax(pred_y, 1)
    match_array = tf.not_equal(pred_y, tr_y)
    indices = tf.where(match_array)
    return tf.transpose(indices)[0]

  def create_model(self, width, height, depth, classes):
    raise NotImplementedError()

  def create_image_generator(self):
    raise NotImplementedError()

  def get_name(self):
    raise NotImplementedError()

In [6]:
# Similar to Alex-net
class alex_net_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "Alex_Net"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=40,
       width_shift_range=0.2,
       height_shift_range=0.2,
       horizontal_flip=True,
       vertical_flip=False)


  def create_model(self, width, height, depth, classes):
    return create_alex_net_architecture(width, height, depth, classes)


In [7]:
# Similar to VGG 16
class vgg_16_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "VGG_16"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=30,
       width_shift_range=0.2,
       height_shift_range=0.2,
       horizontal_flip=True,
       vertical_flip=False)


  def create_model(self, width, height, depth, classes):
    return create_vgg_16_architecture(width, height, depth, classes)

In [8]:
# Case 0
class case_0_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "Case_0"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=20,
       width_shift_range=0.1,
       height_shift_range=0.1,
       horizontal_flip=True,
       vertical_flip=False)


  def create_model(self, width, height, depth, classes):
    return create_case_0_architecture(width, height, depth, classes)

In [9]:
# Case 1 
class case_1_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "Case_1"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=50,
       width_shift_range=0.1,
       height_shift_range=0.1,
       horizontal_flip=True,
       vertical_flip=False)


  def create_model(self, width, height, depth, classes):
    return create_case_1_architecture(width, height, depth, classes)

In [10]:
# Case 2
class case_2_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "Case_2"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=75,
       width_shift_range=0.4,
       height_shift_range=0.4,
       horizontal_flip=True,
       vertical_flip=False,
       zoom_range=0.1)


  def create_model(self, width, height, depth, classes):
    return create_case_2_architecture(width, height, depth, classes)

In [11]:
# Case 3
class case_3_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "Case_3"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=10,
       width_shift_range=0.1,
       height_shift_range=0.1,
       horizontal_flip=True,
       vertical_flip=False,
       zoom_range=0.3)


  def create_model(self, width, height, depth, classes):
    return create_case_3_architecture(width, height, depth, classes)

In [12]:
# Helper function to calculate loss and accuracy
def get_prediction_result(predicted_results, y_labels):
  # Calculate Loss
  y_labels_hot = np_utils.to_categorical(y_labels, NUM_CLASSES)
  predicted_results_clipped = tf.clip_by_value(predicted_results, 1e-10, 1.0)
  loss = (1.0/predicted_results_clipped.shape[0]) * tf.math.reduce_sum(-tf.math.multiply(tf.math.log(predicted_results_clipped),y_labels_hot))

  # Calculate Accuracy
  predicted_y = tf.math.argmax(predicted_results, 1)
  match_array = tf.equal(predicted_y, y_labels)
  accuracy = tf.reduce_sum(tf.cast(match_array, tf.float32)) / len(y_labels)

  return (loss, accuracy)

In [13]:
# Create Ensemble method
# This setup allows incremental training
SHOULD_TRAIN = True       # true if models should be trained
TRAIN_FOR_NUM_EPOCHS = 30 # the number of epochs to run if training
LOAD_WEIGHTS = True       # set to false to restart training

SPLIT_RATIO = 0.85          # the bag size used relative to the whole dataset

# 1. Create the used architectures. This will also load cached best model if exists
print("Initialising CNN architectures")
alex_net   = alex_net_CNN_augmented_checkpointed(LOAD_WEIGHTS, "BOOST")
vgg_16_net = vgg_16_CNN_augmented_checkpointed(LOAD_WEIGHTS, "BOOST")
case_0_net = case_0_CNN_augmented_checkpointed(LOAD_WEIGHTS, "BOOST")
case_1_net = case_1_CNN_augmented_checkpointed(LOAD_WEIGHTS, "BOOST")
case_2_net = case_2_CNN_augmented_checkpointed(LOAD_WEIGHTS, "BOOST")
case_3_net = case_3_CNN_augmented_checkpointed(LOAD_WEIGHTS, "BOOST")


if SHOULD_TRAIN:
  data_size = len(trainX)
  size = int(data_size * SPLIT_RATIO)

  # each model will be trained here. Once training is complete, the checkpointed weights
  # are loaded back in. This is to allow the final tests to be done on the best performing model
  print("Training CNN architectures for {} epochs".format(TRAIN_FOR_NUM_EPOCHS))
  indices = np.random.choice(data_size, size, replace=False)
  # Get the randomly selected data
  split_data_x = trainX[indices]
  split_data_y = trainY[indices]
  print("Alex_Net will train on {} data instances".format(len(split_data_x)))
  # Train
  alex_net.train(split_data_x, split_data_y, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  # Reload best weights
  alex_net.load_checkpointed_weights()
  # Get indices of miss classified instances
  missed_indices = alex_net.get_instances_incorrectly_predicted(split_data_x, split_data_y)

  # Keep the miss classified data in the array
  split_data_x = split_data_x[missed_indices]
  split_data_y = split_data_y[missed_indices]
  num_missed = len(split_data_x)
  print("AlexNet has missed {} data instances".format(num_missed))
  # Randomly choose the rest to fill up the array
  indices = np.random.choice(data_size, (size - num_missed), replace=False)
  split_data_x = np.append(split_data_x, trainX[indices], axis=0)
  split_data_y = np.append(split_data_y, trainY[indices], axis=0)
  # Train
  case_0_net.train(split_data_x, split_data_y, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  # Reload best weights
  case_0_net.load_checkpointed_weights()
  # Get indices of miss classified instances
  missed_indices = case_0_net.get_instances_incorrectly_predicted(split_data_x, split_data_y)

    
  # Keep the miss classified data in the array
  split_data_x = split_data_x[missed_indices]
  split_data_y = split_data_y[missed_indices]
  # Randomly choose the rest to fill up the array
  num_missed = len(split_data_x)
  print("Case_0 has missed {} data instances".format(num_missed))
  indices = np.random.choice(data_size, (size - num_missed), replace=False)
  split_data_x = np.append(split_data_x, trainX[indices], axis=0)
  split_data_y = np.append(split_data_y, trainY[indices], axis=0)
  # Train
  case_1_net.train(split_data_x, split_data_y, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  # Reload best weights
  case_1_net.load_checkpointed_weights()
  # Get indices of miss classified instances
  missed_indices = case_1_net.get_instances_incorrectly_predicted(split_data_x, split_data_y)

  # Keep the miss classified data in the array
  split_data_x = split_data_x[missed_indices]
  split_data_y = split_data_y[missed_indices]
  # Randomly choose the rest to fill up the array
  num_missed = len(split_data_x)
  print("Case_1 has missed {} data instances".format(num_missed))
  indices = np.random.choice(data_size, (size - num_missed), replace=False)
  split_data_x = np.append(split_data_x, trainX[indices], axis=0)
  split_data_y = np.append(split_data_y, trainY[indices], axis=0)
  # Train
  case_2_net.train(split_data_x, split_data_y, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  # Reload best weights
  case_2_net.load_checkpointed_weights()
  # Get indices of miss classified instances
  missed_indices = case_2_net.get_instances_incorrectly_predicted(split_data_x, split_data_y)

    
  # Keep the miss classified data in the array
  split_data_x = split_data_x[missed_indices]
  split_data_y = split_data_y[missed_indices]
  # Randomly choose the rest to fill up the array
  num_missed = len(split_data_x)
  print("Case 2 has missed {} data instances".format(num_missed))
  indices = np.random.choice(data_size, (size - num_missed), replace=False)
  split_data_x = np.append(split_data_x, trainX[indices], axis=0)
  split_data_y = np.append(split_data_y, trainY[indices], axis=0)
  # Train
  vgg_16_net.train(split_data_x, split_data_y, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  # Reload best weights
  vgg_16_net.load_checkpointed_weights()
  # Get indices of miss classified instances
  missed_indices = vgg_16_net.get_instances_incorrectly_predicted(split_data_x, split_data_y)

    
  # Keep the miss classified data in the array
  split_data_x = split_data_x[missed_indices]
  split_data_y = split_data_y[missed_indices]
  # Randomly choose the rest to fill up the array
  num_missed = len(split_data_x)
  print("VGG_16 has missed {} data instances".format(num_missed))
  indices = np.random.choice(data_size, (size - num_missed), replace=False)
  split_data_x = np.append(split_data_x, trainX[indices], axis=0)
  split_data_y = np.append(split_data_y, trainY[indices], axis=0)
  # Train
  case_3_net.train(split_data_x, split_data_y, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  # Reload best weights
  case_3_net.load_checkpointed_weights()

  print("Training CNN architectures complete")

print("Gathering final results")

prediction_a = alex_net.predict(testX)
prediction_v = vgg_16_net.predict(testX)
prediction_0 = case_0_net.predict(testX)
prediction_1 = case_1_net.predict(testX)
prediction_2 = case_2_net.predict(testX)
prediction_3 = case_3_net.predict(testX)

total_predictions = prediction_a + prediction_v + prediction_0 + prediction_1 + prediction_2 + prediction_3

final_loss, final_accuracy = get_prediction_result(total_predictions, testY)

print("Predicting completed. Final Ensemble Loss: {:.4f} Final Ensemble Accuracy: {:.2f}%".format(final_loss, final_accuracy*100.0))

# Print Individual algorithm accuracy
alex_net_loss, alex_net_accuracy          = get_prediction_result(prediction_a, testY)
vgg_16_net_loss, vgg_16_net_accuracy      = get_prediction_result(prediction_v, testY)
case_0_net_loss, case_0_net_accuracy      = get_prediction_result(prediction_0, testY)
case_1_net_loss, case_1_net_accuracy      = get_prediction_result(prediction_1, testY)
case_2_net_loss, case_2_net_accuracy      = get_prediction_result(prediction_2, testY)
case_3_net_loss, case_3_net_accuracy      = get_prediction_result(prediction_3, testY)

print("Alex_net Loss: {:.4f} Accuracy: {:.2f}%".format(alex_net_loss, alex_net_accuracy*100.0))
print("VGG_16_net Loss: {:.4f} Accuracy: {:.2f}%".format(vgg_16_net_loss, vgg_16_net_accuracy*100.0))
print("Case_0_net Loss: {:.4f} Accuracy: {:.2f}%".format(case_0_net_loss, case_0_net_accuracy*100.0))
print("Case_1_net Loss: {:.4f} Accuracy: {:.2f}%".format(case_1_net_loss, case_1_net_accuracy*100.0))
print("Case_2_net Loss: {:.4f} Accuracy: {:.2f}%".format(case_2_net_loss, case_2_net_accuracy*100.0))
print("Case_3_net Loss: {:.4f} Accuracy: {:.2f}%".format(case_3_net_loss, case_3_net_accuracy*100.0))


Initialising CNN architectures
Training CNN architectures for 30 epochs
Alex_Net will train on 867 data instances
Alex_Net - Training for 30 epochs
  ...
    to  
  ['...']
Train for 54 steps, validate on 340 samples
Epoch 1/30
53/54 [============================>.] - ETA: 3s - loss: 2.8320 - accuracy: 0.0719
Alex_Net - Caching Checkpoint - val_loss has improved from: inf to: 2.8280
54/54 [==============================] - 220s 4s/step - loss: 2.8319 - accuracy: 0.0717 - val_loss: 2.8280 - val_accuracy: 0.0765
Epoch 2/30
53/54 [============================>.] - ETA: 2s - loss: 2.8229 - accuracy: 0.0898
Alex_Net - Caching Checkpoint - val_loss has improved from: 2.8280 to: 2.8138
54/54 [==============================] - 180s 3s/step - loss: 2.8228 - accuracy: 0.0905 - val_loss: 2.8138 - val_accuracy: 0.1147
Epoch 3/30
53/54 [============================>.] - ETA: 3s - loss: 2.7952 - accuracy: 0.1293
Alex_Net - Caching Checkpoint - val_loss has improved from: 2.8138 to: 2.7618
54/54 [===

Epoch 28/30
53/54 [============================>.] - ETA: 3s - loss: 1.7121 - accuracy: 0.3832
Alex_Net Val_loss did not improve. Best is: 1.5569. Not Caching
54/54 [==============================] - 189s 4s/step - loss: 1.7124 - accuracy: 0.3819 - val_loss: 1.6191 - val_accuracy: 0.4118
Epoch 29/30
53/54 [============================>.] - ETA: 3s - loss: 1.6576 - accuracy: 0.4048
Alex_Net - Caching Checkpoint - val_loss has improved from: 1.5569 to: 1.5512
54/54 [==============================] - 191s 4s/step - loss: 1.6585 - accuracy: 0.4078 - val_loss: 1.5512 - val_accuracy: 0.4735
Epoch 30/30
53/54 [============================>.] - ETA: 3s - loss: 1.6237 - accuracy: 0.4132
Alex_Net - Caching Checkpoint - val_loss has improved from: 1.5512 to: 1.4700
54/54 [==============================] - 192s 4s/step - loss: 1.6261 - accuracy: 0.4101 - val_loss: 1.4700 - val_accuracy: 0.4559
Alex_Net - Loading previous training data
Alex_Net - Loaded previous model weights from: C:\Users\danie\D

Epoch 24/30
53/54 [============================>.] - ETA: 0s - loss: 1.0539 - accuracy: 0.6431
Case_0 Val_loss did not improve. Best is: 1.1456. Not Caching
54/54 [==============================] - 38s 712ms/step - loss: 1.0690 - accuracy: 0.6381 - val_loss: 1.2422 - val_accuracy: 0.5941
Epoch 25/30
53/54 [============================>.] - ETA: 0s - loss: 1.0376 - accuracy: 0.6228
Case_0 Val_loss did not improve. Best is: 1.1456. Not Caching
54/54 [==============================] - 38s 713ms/step - loss: 1.0385 - accuracy: 0.6228 - val_loss: 1.1642 - val_accuracy: 0.6265
Epoch 26/30
53/54 [============================>.] - ETA: 0s - loss: 1.0082 - accuracy: 0.6551
Case_0 Val_loss did not improve. Best is: 1.1456. Not Caching
54/54 [==============================] - 42s 785ms/step - loss: 1.0053 - accuracy: 0.6569 - val_loss: 1.1721 - val_accuracy: 0.6059
Epoch 27/30
53/54 [============================>.] - ETA: 0s - loss: 0.9750 - accuracy: 0.6479
Case_0 Val_loss did not improve. Best 

Epoch 20/30
53/54 [============================>.] - ETA: 0s - loss: 1.2693 - accuracy: 0.5557
Case_1 Val_loss did not improve. Best is: 1.2084. Not Caching
54/54 [==============================] - 31s 575ms/step - loss: 1.2708 - accuracy: 0.5558 - val_loss: 1.2849 - val_accuracy: 0.5471
Epoch 21/30
53/54 [============================>.] - ETA: 0s - loss: 1.2139 - accuracy: 0.5665
Case_1 - Caching Checkpoint - val_loss has improved from: 1.2084 to: 1.1485
54/54 [==============================] - 31s 581ms/step - loss: 1.2196 - accuracy: 0.5617 - val_loss: 1.1485 - val_accuracy: 0.6441
Epoch 22/30
53/54 [============================>.] - ETA: 0s - loss: 1.2342 - accuracy: 0.5485
Case_1 - Caching Checkpoint - val_loss has improved from: 1.1485 to: 1.1149
54/54 [==============================] - 32s 584ms/step - loss: 1.2398 - accuracy: 0.5476 - val_loss: 1.1149 - val_accuracy: 0.6382
Epoch 23/30
53/54 [============================>.] - ETA: 0s - loss: 1.1907 - accuracy: 0.5605
Case_1 - C

Epoch 16/30
53/54 [============================>.] - ETA: 0s - loss: 2.1941 - accuracy: 0.2108
Case_2 Val_loss did not improve. Best is: 2.0394. Not Caching
54/54 [==============================] - 29s 537ms/step - loss: 2.1942 - accuracy: 0.2127 - val_loss: 2.0415 - val_accuracy: 0.2235
Epoch 17/30
53/54 [============================>.] - ETA: 0s - loss: 2.1905 - accuracy: 0.2467
Case_2 - Caching Checkpoint - val_loss has improved from: 2.0394 to: 1.9842
54/54 [==============================] - 29s 531ms/step - loss: 2.1945 - accuracy: 0.2456 - val_loss: 1.9842 - val_accuracy: 0.2647
Epoch 18/30
53/54 [============================>.] - ETA: 0s - loss: 2.1363 - accuracy: 0.2683
Case_2 Val_loss did not improve. Best is: 1.9842. Not Caching
54/54 [==============================] - 29s 532ms/step - loss: 2.1366 - accuracy: 0.2656 - val_loss: 2.1082 - val_accuracy: 0.2324
Epoch 19/30
53/54 [============================>.] - ETA: 0s - loss: 2.1119 - accuracy: 0.2599
Case_2 - Caching Checkpo

53/54 [============================>.] - ETA: 3s - loss: 2.6429 - accuracy: 0.1425
VGG_16 Val_loss did not improve. Best is: 2.5376. Not Caching
54/54 [==============================] - 221s 4s/step - loss: 2.6469 - accuracy: 0.1410 - val_loss: 2.8545 - val_accuracy: 0.0471
Epoch 13/30
53/54 [============================>.] - ETA: 3s - loss: 2.8147 - accuracy: 0.0766
VGG_16 Val_loss did not improve. Best is: 2.5376. Not Caching
54/54 [==============================] - 221s 4s/step - loss: 2.8143 - accuracy: 0.0764 - val_loss: 2.8569 - val_accuracy: 0.0471
Epoch 14/30
53/54 [============================>.] - ETA: 3s - loss: 2.8015 - accuracy: 0.0814
VGG_16 Val_loss did not improve. Best is: 2.5376. Not Caching
54/54 [==============================] - 229s 4s/step - loss: 2.8032 - accuracy: 0.0811 - val_loss: 2.8602 - val_accuracy: 0.0441
Epoch 15/30
53/54 [============================>.] - ETA: 4s - loss: 2.7898 - accuracy: 0.1018
VGG_16 Val_loss did not improve. Best is: 2.5376. Not Ca

53/54 [============================>.] - ETA: 0s - loss: 2.3690 - accuracy: 0.1851
Case_3 - Caching Checkpoint - val_loss has improved from: 2.4823 to: 2.2720
54/54 [==============================] - 33s 612ms/step - loss: 2.3685 - accuracy: 0.1852 - val_loss: 2.2720 - val_accuracy: 0.1824
Epoch 9/30
53/54 [============================>.] - ETA: 0s - loss: 2.3624 - accuracy: 0.1952
Case_3 - Caching Checkpoint - val_loss has improved from: 2.2720 to: 2.2459
54/54 [==============================] - 32s 601ms/step - loss: 2.3623 - accuracy: 0.1939 - val_loss: 2.2459 - val_accuracy: 0.1765
Epoch 10/30
53/54 [============================>.] - ETA: 0s - loss: 2.2603 - accuracy: 0.2084
Case_3 - Caching Checkpoint - val_loss has improved from: 2.2459 to: 2.1778
54/54 [==============================] - 33s 612ms/step - loss: 2.2609 - accuracy: 0.2056 - val_loss: 2.1778 - val_accuracy: 0.1941
Epoch 11/30
53/54 [============================>.] - ETA: 0s - loss: 2.1758 - accuracy: 0.2488
Case_3 - 